In [1]:
!pip install pyrosettacolabsetup
import pyrosettacolabsetup; pyrosettacolabsetup.install_pyrosetta()
from pyrosetta import *
from pyrosetta.rosetta import *
from pyrosetta.teaching import *
import pyrosetta.toolbox
#pyrosetta.init()

pyrosetta.init("-ex1 -ex2aro")
!pip install py3Dmol
import py3Dmol

Mounted at /content/google_drive

Note that USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRE PURCHASE OF A LICENSE.
See https://github.com/RosettaCommons/rosetta/blob/main/LICENSE.md or email license@uw.edu for details.

Looking for compatible PyRosetta wheel file at google-drive/PyRosetta/colab.bin//wheels...
Found compatible wheel: /content/google_drive/MyDrive/PyRosetta/colab.bin/wheels//content/google_drive/MyDrive/PyRosetta/colab.bin/wheels/pyrosetta-2024.19+release.a34b73c40f-cp310-cp310-linux_x86_64.whl


┌──────────────────────────────────────────────────────────────────────────────┐
│                                 PyRosetta-4                                  │
│              Created in JHU by Sergey Lyskov and PyRosetta Team              │
│              (C) Copyright Rosetta Commons Member Institutions               │
│                                                                              │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRE PURCHASE OF A LICENS

In [4]:
# Load the Trp-cage structure (PDB ID: 1L2Y)
pose = rosetta.core.import_pose.pose_from_file("/content/google_drive/MyDrive/BIP_24-25/practicas/Trp-Cage_miniprotein/12LY.pdb")

# Output initial pose information
scorefxn = pyrosetta.create_score_function("ref2015_cart.wts")

score = scorefxn(pose)
print("Initial pose energies:")
print(pose.energies())

core.import_pose.import_pose: File '/content/google_drive/MyDrive/BIP_24-25/practicas/Trp-Cage_miniprotein/12LY.pdb' automatically determined to be of type PDB
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 3 atoms at position 1 in file /content/google_drive/MyDrive/BIP_24-25/practicas/Trp-Cage_miniprotein/12LY.pdb. Best match rsd_type:  ASN:NtermProteinFull
core.energy_methods.CartesianBondedEnergy: Creating new peptide-bonded energy container (20)
basic.io.database: Database file opened: scoring/score_functions/elec_cp_reps.dat
core.scoring.elec.util: Read 40 countpair representative atoms
core.pack.dunbrack.RotamerLibrary: shapovalov_lib_fixes_enable option is true.
core.pack.dunbrack.RotamerLibrary: shapovalov_lib::shap_dun10_smooth_level of 1( aka lowest_smooth ) got activated.
core.pack.dunbrack.RotamerLibrary: Binary rotamer library selected: /usr/local/lib/python3.10/dist-packages/pyrosetta/database/rotamer/shapovalov/StpDwn_0-0-0/Dunbrack10.lib.bin
core.pack.

In [5]:
# Setting up the packer task to allow mutations at core residues
task_factory = rosetta.core.pack.task.TaskFactory()

# Add TaskOperations for design
restrict_to_repacking = rosetta.core.pack.task.operation.RestrictToRepacking()
task_factory.push_back(restrict_to_repacking)

# Load a resfile that specifies mutations in the core (students can modify this)
resfile = """
NATRO
start
8 A PIKAA L
9 A PIKAA L
10 A PIKAA L
11 A PIKAA L
16 A PIKAA W
18 A PIKAA W
20 A PIKAA W
"""
# Save the resfile for the TaskOperation
with open("resfile.txt", "w") as file:
    file.write(resfile)

# Create a task operation from the resfile
resfile_reader = rosetta.core.pack.task.operation.ReadResfile()
resfile_reader.filename("resfile.txt")
task_factory.push_back(resfile_reader)

# Set up the FastRelax protocol
relax = rosetta.protocols.relax.FastRelax()

protocols.relax.RelaxScriptManager: Reading relax scripts list from database.
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
protocols.relax.RelaxScriptManager: Looking for MonomerRelax2019.txt
protocols.relax.RelaxScriptManager: ================== Reading script file: /usr/local/lib/python3.10/dist-packages/pyrosetta/database/sampling/relax_scripts/MonomerRelax2019.txt ==================
protocols.relax.RelaxScriptManager: repeat %%nrepeats%%
protocols.relax.RelaxScriptManager: coord_cst_weight 1.0
protocols.relax.RelaxScriptManager: scale:fa_rep 0.040
protocols.relax.RelaxScriptManager: repack
protocols.relax.RelaxScriptManager: scale:fa_rep 0.051
protocols.relax.RelaxScriptManager: min 0.01
protocols.relax.RelaxScriptManager: coord_cst_weight 0.5
protocols.relax.RelaxScriptManager: scale:fa_rep 0.265
protocols.relax.RelaxScriptManager: repack
protocols.relax.RelaxScriptManager: scale:fa_rep 0.280
protocols.relax.RelaxScriptManager: min 0.01
protocols.relax.RelaxScriptMana

In [6]:
# Attach the score function to FastRelax
relax.set_scorefxn(scorefxn)

# Create a packer task for design with mutations
packer_task = task_factory.create_task_and_apply_taskoperations(pose)

# Apply FastRelax to the pose with the allowed mutations from the resfile
relax.apply(pose)

# Output the resulting pose information after FastRelax
print("Pose energies after FastRelax:")
print(pose.energies())

protocols.relax.FastRelax: CMD: repeat  55.7971  0  0  0.55
protocols.relax.FastRelax: CMD: coord_cst_weight  55.7971  0  0  0.55
protocols.relax.FastRelax: CMD: scale:fa_rep  47.4479  0  0  0.022
core.pack.task: Packer task: initialize from command line()
core.pack.pack_rotamers: built 281 rotamers at 20 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
protocols.relax.FastRelax: CMD: repack  -19.7956  0  0  0.022
protocols.relax.FastRelax: CMD: scale:fa_rep  -19.3782  0  0  0.02805
protocols.relax.FastRelax: CMD: min  -39.027  0.458705  0.458705  0.02805
protocols.relax.FastRelax: CMD: coord_cst_weight  -39.027  0.458705  0.458705  0.02805
protocols.relax.FastRelax: CMD: scale:fa_rep  -23.1737  0.458705  0.458705  0.14575
core.pack.task: Packer task: initialize from command line()
core.pack.pack_rotamers: built 253 rotamers at 20 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGrap

In [8]:
# Calculate the total score and display the energy changes
initial_score = scorefxn(pose)
print(f"Initial total score: {initial_score}")

# Saving the designed protein to a PDB file
pose.dump_pdb("stabilized_trpcage.pdb")

# Clean up (delete the resfile)
os.remove("resfile.txt")

print("Design complete. Stabilized mutant saved as stabilized_trpcage.pdb")


Initial total score: -33.06071533821303


FileNotFoundError: [Errno 2] No such file or directory: 'resfile.txt'

In [ ]:
# Import PyRosetta
import pyrosetta
from pyrosetta import rosetta
import os

# Initialize PyRosetta
pyrosetta.init()

# Load the Trp-cage structure (PDB ID: 1L2Y)
pose = rosetta.core.import_pose.pose_from_file("1L2Y.pdb")

# Output initial pose information
print("Initial pose energies:")
print(pose.energies())

# Setting up the packer task to allow mutations at core residues
task_factory = rosetta.core.pack.task.TaskFactory()

# Add TaskOperations for design
restrict_to_repacking = rosetta.core.pack.task.operation.RestrictToRepacking()
task_factory.push_back(restrict_to_repacking)

# Load a resfile that specifies mutations in the core (students can modify this)
resfile = """
NATRO
start
8 A PIKAA L
9 A PIKAA L
10 A PIKAA L
11 A PIKAA L
16 A PIKAA W
18 A PIKAA W
20 A PIKAA W
"""
# Save the resfile for the TaskOperation
with open("resfile.txt", "w") as file:
    file.write(resfile)

# Create a task operation from the resfile
resfile_reader = rosetta.core.pack.task.operation.ReadResfile()
resfile_reader.filename("resfile.txt")
task_factory.push_back(resfile_reader)

# Set up the FastRelax protocol
relax = rosetta.protocols.relax.FastRelax()

# Set up the score function (standard score function for FastRelax)
scorefxn = rosetta.get_fa_scorefxn()

# Attach the score function to FastRelax
relax.set_scorefxn(scorefxn)

# Create a packer task for design with mutations
packer_task = task_factory.create_task_and_apply_taskoperations(pose)

# Apply FastRelax to the pose with the allowed mutations from the resfile
relax.apply(pose)

# Output the resulting pose information after FastRelax
print("Pose energies after FastRelax:")
print(pose.energies())

# Calculate the total score and display the energy changes
initial_score = scorefxn(pose)
print(f"Initial total score: {initial_score}")

# Saving the designed protein to a PDB file
pose.dump_pdb("stabilized_trpcage.pdb")

# Clean up (delete the resfile)
os.remove("resfile.txt")

print("Design complete. Stabilized mutant saved as stabilized_trpcage.pdb")
